In [ ]:
# Define conditions
dp_filters = [0, 10, 20]
vaf_filters = [0, 25]
variant_percentages = [0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 1.00]

# Read data
genotype_counts = read_vcf_data("xhetrel/merged.vcf.gz")
sex_df = read_sample_sex("xhetrel/sample_sex.tsv")

# Analyze each condition
results = {}
for dp in dp_filters:
    for vaf in vaf_filters:
        depth_range = f"DP{dp}_VAF{vaf}"
        print(f"\nAnalyzing {depth_range}")

        # Filter variants
        filtered_counts = filter_variants(genotype_counts, dp, vaf)

        # Calculate initial heterozygosity ratios
        het_ratios_df = calculate_het_ratios(filtered_counts, sex_df)

        # Analyze with different variant percentages
        results[depth_range] = {}
        for pct in variant_percentages:
            print(f"  {pct*100:.0f}% variants")
            metrics, het_ratios_df = analyze_condition(het_ratios_df, depth_range, pct)
            results[depth_range][pct] = {
                **metrics,
                'het_ratios_df': het_ratios_df
            }

# Write results
write_results(results)

<span class="codehilite"><div class="highlight"><pre><span></span><span class="gt">Traceback (most recent call last):</span>
  File <span class="nb">&quot;/home/bar/Desktop/Workbench/phd/Analizler/.venv/lib64/python3.13/site-packages/marimo/_runtime/executor.py&quot;</span>, line <span class="m">138</span>, in <span class="n">execute_cell</span>
<span class="w">    </span><span class="n">exec</span><span class="p">(</span><span class="n">cell</span><span class="o">.</span><span class="n">body</span><span class="p">,</span> <span class="n">glbls</span><span class="p">)</span>
<span class="w">    </span><span class="pm">~~~~^^^^^^^^^^^^^^^^^^</span>
  File <span class="nb">&quot;/tmp/marimo_805384/__marimo__cell_Hbol_.py&quot;</span>, line <span class="m">7</span>, in <span class="n">&lt;module&gt;</span>
<span class="w">    </span><span class="n">genotype_counts</span> <span class="o">=</span> <span class="n">read_vcf_data</span><span class="p">(</span><span class="s2">&quot;xhetrel/mer